# Different noise models

In this notebook we look at the different noise models.

## Prepare train, test and side-info

We first need to download and prepare the data files. This can be acomplished using this a built-in function is smurff. IC50 is a compound x protein matrix, The ECFP matrix as features as side information on the compounds.

In [ ]:
import smurff

ic50_train, ic50_test, ecfp = smurff.load_chembl()

## Fixed noise

The noise model of observed data  can be annotated by calling `addTrainAndTest` with
the optional parameter `noise_model`. The default for this parameter is `FixedNoise` with precision 5.0

In [ ]:
session = smurff.TrainSession(
                            priors = ['normal', 'normal'],
                            num_latent=32,
                            burnin=100,
                            nsamples=500)

# the following line is equivalent to the default, not specifing noise_model
session.addTrainAndTest(ic50_train, ic50_test, smurff.FixedNoise(5.0))
predictions = session.run()
print("RMSE = %.2f" % smurff.call_rmse(predictions))

## Adaptive noise

Instead of a fixed precision, we can also allow the model to automatically determine the precision of the noise by using `AdaptiveNoise`, with signal-to-noise ratio parameters `sn_init` and `sn_max`.

- `sn_init` is an initial signal-to-noise ratio.
- `sn_max`  is the maximum allowed signal-to-noise ratio. This means that if the updated precision would imply a higher signal-to-noise ratio than `sn_max`, then the precision value is set to `(sn_max + 1.0) / Yvar` where `Yvar` is the variance of the training dataset `Y`.

In [ ]:
session = smurff.TrainSession(
                            priors = ['normal', 'normal'],
                            num_latent=32,
                            burnin=100,
                            nsamples=500)

session.addTrainAndTest(ic50_train, ic50_test, smurff.AdaptiveNoise(1.0, 10.))
predictions = session.run()
print("RMSE = %.2f" % smurff.call_rmse(predictions))

## Binary matrices

SMURFF can also factorize binary matrices (with or without side information).
The input matrices can contain arbitrary values, and are converted to 0's and 1' by means of a threshold.
To factorize them we employ probit noise model `ProbitNoise`, taking this
`threshold` as a parameter.

To evaluate binary factorization, we recommed to use *ROC AUC*, which can be enabled by providing
a threshold also to the `TrainsSession`.

In [ ]:
ic50_threshold = 6.
session = smurff.TrainSession(
                            priors = ['normal', 'normal'],
                            num_latent=32,
                            burnin=100,
                            nsamples=100,
                            # Using threshold of 6. to calculate AUC on test data
                            threshold=ic50_threshold)

## using activity threshold pIC50 > 6. to binarize train data
session.addTrainAndTest(ic50_train, ic50_test, smurff.ProbitNoise(ic50_threshold))
predictions = session.run()
print("RMSE = %.2f" % smurff.calc_rmse(predictions))
print("AUC = %.2f" % smurff.calc_auc(predictions, ic50_threshold))